In [2]:
%pip install -q ipywidgets

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate
from scipy.stats import lognorm
from scipy.optimize import fsolve
from scipy.integrate import simpson

from ipywidgets import interactive, fixed

Matplotlib is building the font cache; this may take a moment.


In [4]:
def adjust_bateman_param(Tmax, Cmax, T_half, m):
    # Calculate elimination rate constant ke
    ke = np.log(2) / T_half
    ke_adj = ke * m  # Adjust ke based on metabolism rate

    # Adjust Tmax based on metabolism rate
    Tmax_adj = Tmax / m

    # Adjust Cmax based on metabolism rate
    Cmax_adj = Cmax / np.sqrt(m)

    # Estimate absorption rate constant ka from Tmax and ke_adj
    def equation(ka):
        return (np.log(ka) - np.log(ke_adj)) / (ka - ke_adj) - Tmax_adj

    ka_initial_guess = 0.1
    ka_adj = fsolve(equation, ka_initial_guess, maxfev=10000)[0]  # Adjust ka based on metabolism rate

    # Calculate adjusted half-life
    T_half_adj = np.log(2) / ke_adj

    # Calculate the constant A (proportionality constant) from Cmax_adj
    A_adj = Cmax_adj / (np.exp(-ke_adj * Tmax_adj) - np.exp(-ka_adj * Tmax_adj))

    return (Tmax_adj, Cmax_adj, T_half_adj, A_adj, ke_adj, ka_adj)


In [5]:
def float_to_time_str_5min(f):
    total_minutes = int(round(f * 60 / 5) * 5)
    hours = total_minutes // 60
    minutes = total_minutes % 60
    return f"{hours:02d}:{minutes:02d}"


In [12]:
def solve_med(Tmax=2, Cmax=80, T_half=21, metabolism=0.75, dose_interval=6, dose_per_day=1, skip_after_period=0, days=1):
    # Given parameters
    # Tmax = 1  # hours
    # Cmax = 100  # ng/mL
    # Thalf = 5  # hours
    # dose_interval = 24  # hours between doses
    # num_doses = 14     # number of doses to simulate
    first_dose_hour = 8

    (Tmax, Cmax, T_half, A, ke, ka) = adjust_bateman_param(Tmax, Cmax, T_half, metabolism)

    # Time points for plotting
    total_time = 4 * T_half + 24 * days  # total simulation time
    data_points = dose_per_day*days*1000
    time = np.linspace(0, total_time, data_points)

    # Bateman function for single dose at time t0
    def bateman(t, t0=0):
        dt = t - t0
        dt = np.maximum(dt, 0)  # concentration is zero before dose
        return A * (np.exp(-ke * dt) - np.exp(-ka * dt))

    # Sum concentrations from multiple doses
    concentration = np.zeros_like(time)
    for d in range(days):
        if d > 0 and skip_after_period > 0 and d % (skip_after_period+1) == skip_after_period: 
            continue
        first_dose_time = 24 * d + first_dose_hour
        for n in range(dose_per_day):
            dose_time = n * dose_interval
            assert dose_time <= 24
            dose_time = first_dose_time + dose_time
            concentration += bateman(time, t0=dose_time)

    day_points_num = int(float(24 * days) / total_time * data_points)
    avg_conc = np.average(concentration[0:day_points_num])

    area = simpson(concentration, x=time)

    max_conc = np.max(concentration)
    Tmax_real = time[np.argmax(concentration)]

    print(f"Adjust for metabolism: Tmax={Tmax}, Cmax={Cmax}, T_half={T_half}")
    print(f"Adjust for metabolism: A={A}, ke={ke}, ka={ka}")
    print("Average (ng/ml):", avg_conc)
    print("Average at Bedtime (ng/ml):", avg_conc)
    print("Plasma Volume AUC per Day (ng/ml):", area/(total_time/24.0))

    # Plotting
    fig, ax = plt.subplots(figsize=(20, 12))

    title_info = f"{days}-Day Administration, {dose_per_day}-Time Daily Dosing Starts {float_to_time_str_5min(first_dose_hour)}"
    if dose_per_day > 1: 
        title_info += f" with {dose_interval}-Hour Interval"
    if skip_after_period > 0 and d > skip_after_period: 
        title_info += f", Skip for 1 Day After {skip_after_period}-Day Period"

    ax.plot(time, concentration, label='Plasma Concentration Level')
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Plasma Concentration (ng/mL)')
    ax.set_title('Plasma Concentration-Time Profile (Bateman Function)\n' + title_info)

    tick_positions = np.arange(0, total_time, 8)
    tick_labels = [str(i//3) if i % 3 == 0 else '' for i in range(len(tick_positions))]

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_xlabel('Days')
    ax.margins(x=0)


    if days == 1:
        # Add text label left of the left edge of the horizontal line
        # Get current x-axis limits
        y_min, y_max = ax.get_ylim()
        # Place text slightly left of the left edge of the horizontal line
        y_text = y_min + (y_max - y_min) * 0.01  # 2% left of x_min
        # Add horizontal line at overall max concentration
        ax.axvline(x=Tmax_real, color='y', linestyle='--', linewidth=1.5, label='Tmax')
        ax.text(
            x=Tmax_real, y=y_text,
            s=float_to_time_str_5min(Tmax_real),
            color='y',
            va='center',
            ha='left',
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.9, edgecolor='none', pad=0)
        )

    # Add text label left of the left edge of the horizontal line
    # Get current x-axis limits
    x_min, x_max = ax.get_xlim()
    # Place text slightly left of the left edge of the horizontal line
    x_text = x_min + (x_max - x_min) * 0.005  # 2% left of x_min
    # Add horizontal line at overall max concentration
    max_label = 'Cmax' if dose_per_day == 1 and days == 1 else f'Max within {days}d'
    ax.axhline(y=max_conc, color='red', linestyle='--', linewidth=1.5, label=max_label)
    ax.text(
        x=x_text, y=max_conc,
        s=f'{max_conc:.2f}',
        color='red',
        va='center',
        ha='left',
        fontsize=10,
        bbox=dict(facecolor='white', alpha=0.9, edgecolor='none', pad=0)
    )


    # Add text label left of the left edge of the horizontal line
    # Get current x-axis limits
    x_min, x_max = ax.get_xlim()
    # Place text slightly left of the left edge of the horizontal line
    x_text = x_min + (x_max - x_min) * 0.005  # 2% left of x_min
    # Add horizontal line at overall max concentration
    ax.axhline(y=avg_conc, color='g', linestyle='--', linewidth=1.5, label=f'Average within {days}d')
    ax.text(
        x=x_text, y=avg_conc,
        s=f'{avg_conc:.2f}',
        color='g',
        va='center',
        ha='left',
        fontsize=10,
        bbox=dict(facecolor='white', alpha=0.9, edgecolor='none', pad=0)
    )

    xgridlines = ax.get_xgridlines()
    for i, gridline in enumerate(xgridlines):
        if i % 3 == 0:
            gridline.set_linewidth(1.5)
        else:
            gridline.set_linewidth(0.5)

    ax.set_ylim(top=350)

    ax.legend()
    ax.grid(True)
    plt.show()



In [13]:
w=interactive(solve_med,Tmax=(0.1,10),Cmax=(10.0,180), T_half=(1.0,30), metabolism=(0.50,2.00, 0.05), dose_interval=(0.5,16), dose_per_day=(1,24), skip_after_period = (0, 10), days = (1,56))
w

interactive(children=(FloatSlider(value=2.0, description='Tmax', max=10.0, min=0.1), FloatSlider(value=80.0, d…

# Bupropion Data

| Type | Dose\|mg | Cmax\|ng/ml | Tmax\|h | T_half\|h |   Dosing  | Max\|14d | Average\|14d | Integral/d\|14d |
|:----:|:--------:|:-----------:|:-------:|:---------:|:---------:|:--------:|:------------:|:---------------:|
|  IR  |   37.5   |      40     |   1.5   |     21    | 2/d, q.8h |    208   |      157     |       3178      |
|  IR  |    75    |      80     |   1.5   |     21    |    1/d    |    226   |      159     |       3180      |
|  IR  |    150   |     160     |   1.5   |     21    |    1/d    |    453   |      319     |       6361      |
|  SR  |    100   |      85     |    3    |     21    |    1/d    |    224   |      178     |       3550      |
|  SR  |    150   |     130     |    3    |     21    |    1/d    |    373   |      272     |       5430      |
|  XL  |    150   |     120     |    5    |     21    |    1/d    |    353   |      267     |       5353      |
|  XL  |    150   |     120     |    5    |     21    |    1/2d   |    218   |      138     |       2682      |
|  XL  |    150   |     120     |    5    |     21    |    3/4d   |    318   |      211     |       4164      |